In questa sezione ci occupiamo di analizzare le statistiche dei portieri e capire le correlazione fra queste.

In [1]:
# Importo le librerie necessarie per il codice

import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
pd.set_option('display.max_columns', None)

# Elenco dei file da leggere
files = ['portieri_23.xlsx', 'portieri_22.xlsx', 'portieri_21.xlsx', 'portieri_20.xlsx', 'portieri_19.xlsx']

# Faccio un dizionario per mappare i file agli anni
year_mapping = {'portieri_23.xlsx': 2023,'portieri_22.xlsx': 2022,'portieri_21.xlsx': 2021,'portieri_20.xlsx': 2020,'portieri_19.xlsx': 2019}

Eliminiamo prima le colonne non utili a questa analisi. Iniziamo da quelle di Nome, Nazionalità, Posizione e Squadra. Dopodichè togliamo le colonne con le percentuali e la colonna relativa al numero di rigori subiti dal portiere, in quanto ci interessa di più quanti ne salva o ne concede piuttosto che quelli che gli vengono calciati contro.

In [ ]:
# Questa parte di codice crea 5 figure che rappresentano il riquadro di correlazione fra le statistiche
# suddiviso per annata
colonne_da_eliminare = ['Player', 'Nation', 'Pos', 'Squad', 'Save%', 'CS%', 'PKatt','Save%.1','AvgLen', 'AvgLen.1',
                         'Cmp%', 'Launch%', 'Launch%.1', 'Stp%']

correlations = []
years = []
for file in files:
    
    data_frame = pd.read_excel(file)

    # Uso .drop per rimuovere dal data frame quelle statistiche che non voglio usare in questa analisi
    data_frame.drop(columns=colonne_da_eliminare, inplace=True) 

    corr_matrix = data_frame.corr()  # Calcolo la matrice di correlazione

    year = year_mapping[file]  # Ottengo l'anno dal file usando il mapping
    years.append(year)

    fig = px.imshow(corr_matrix, title=f"Matrice di Correlazione - Anno {year}")
    # Ottengo in output il grafico di correlazione

    fig.show()

# Inserisco questa riga di codice in modo da poter visualizzare le colonne usate per l'analisi
print(f'Le colonne usate per questa analisi sono {list(data_frame.columns)}')


In [3]:
# Creo una funzione che mi permette di scoprire quali sono le statistiche meglio e peggio correlate con una desiderata

def best_worst(stat):
 for file in files:
    year = year_mapping[file]
    data_frame = pd.read_excel(file)

    data_frame.drop(columns=colonne_da_eliminare, inplace=True) 

    corr_matrix = data_frame.corr()

    if stat in corr_matrix.columns:
        # Ordina le correlazioni con la statistica scelta
        sorted_corr = corr_matrix[stat].sort_values(ascending=False)

        # Trova le 5 migliori e peggiori correlazioni (cambiare i range per saperne di più o di meno)
        best_corr = sorted_corr[1:6]  # Parto da 1 per evitare la correlazione della statistica con se stessa
        worst_corr = sorted_corr[-5:]

        print(f"Anno {year}: Le 5 statistiche meglio correlate con {stat} sono:\n{best_corr}")
        print(f"Anno {year}: Le 5 statistiche peggio correlate con {stat} sono:\n{worst_corr}")
    else:
        print(f"La statistica {stat} non è presente nel dataset. Controlla l'input")

# Creo una funzione che restituisce l'elenco degli indci di correlazione di una statistica con le altre anno per anno

def correlazioni_annuali(stat):
    for file in files:
        year = year_mapping[file]
        data_frame = pd.read_excel(file)  

        data_frame.drop(columns=colonne_da_eliminare, inplace=True)

        corr_matrix = data_frame.corr()

        if stat in corr_matrix.columns:
            sorted_corr = corr_matrix[stat].sort_values(ascending=False)

            print(f"Anno {year}: Le correlazioni per {stat} sono:\n{sorted_corr}\n")
        else:
            print(f"La statistica {stat} non è presente nel dataset dell'anno {year}.")
        
# Creo una seconda funzione che mi dice l'indice di correlazione di due statistiche specifiche
# ed il cambiamento di questo nel corso delle stagioni

def correlazione(stat1,stat2):
 correlations = []

 for file in files:
    
    data_frame = pd.read_excel(file)

    data_frame.drop(columns=colonne_da_eliminare, inplace=True)

    corr_matrix = data_frame.corr()

    year = year_mapping[file]

    # Verifica se entrambe le statistiche esistono nella matrice di correlazione
    if stat1 in corr_matrix.columns and stat2 in corr_matrix.columns:
        correlation_value = corr_matrix.loc[stat1, stat2]
        correlations.append((year, correlation_value))
        #print(f"Anno {year}: Correlazione tra {stat1} e {stat2} = {correlation_value}")
    else:
        correlations.append((year, None))

# Creazione del DataFrame per il grafico
 if stat1 in corr_matrix.columns and stat2 in corr_matrix.columns:
    correlation_df = pd.DataFrame(correlations, columns=['Anno', 'Correlazione'])
    
    # Crea il grafico dell'andamento della correlazione nel tempo
    fig = px.line(correlation_df, x='Anno', y='Correlazione', title=f"Correlazione tra {stat1} e {stat2}", markers=True)
    fig.update_xaxes(dtick=1)
    fig.update_yaxes(range = [-1,1], dtick = 0.10)
    fig.show()
 else:
    print("Nessun grafico disponibile: non sono state calcolate correlazioni valide. Controlla l'input")

Le statistiche di gioco si suddividono in 2 macro categorie. Una relativa alle parate o agli interventi realizzati dal portiere, l'altra invece descrive il comportamento del portiere con la palla ai piedi (statistiche su lanci o passaggi).

Iniziamo dalla prima, quella delle parate, ed iniziamo l'analisi con l'età, i minuti e le partite giocate.

In [4]:
colonne_da_eliminare = ['Player', 'Nation', 'Pos', 'Squad','Save%', 'AvgLen', 'AvgLen.1', 'CS%', 'Save%.1', 'Cmp', 'Att', 'Cmp%', 'Att (GK)', 'Thr', 'Launch%', 
'AvgLen', 'Att.1', 'Launch%.1', 'AvgLen.1', 'Opp', 'Stp', 'Stp%']

In [ ]:
correlations = []
years = []
for file in files:
    
    data_frame = pd.read_excel(file)

    data_frame.drop(columns=colonne_da_eliminare, inplace=True) 

    corr_matrix = data_frame.corr() 

    year = year_mapping[file] 
    years.append(year)

    fig = px.imshow(corr_matrix, title=f"Matrice di Correlazione - Anno {year}")

    fig.show()

Il dato sull'età è correlato molto male, gli indici sono tutti vicini allo zero. Risultato abbastanza inaspettato per una posizione del genere, infatti quello del portiere è il ruolo meno fisico e quindi in cui l'età influisce meno a livello di prestazione. Inoltre ci aspettavamo che l'età fosse molto importante, in quanto un portiere più vecchio ha probabilmente più esperienza e più leadership. 
I dati sulle partite giocate è invece abbastanza chiaro in quanto è nell'interesse della squadra far giocare titolare il miglior portiere che ha in rosa.

In [6]:
colonne_da_eliminare = ['Player', 'Nation', 'Pos', 'Squad','Save%', 'Age', 'Starts', 'MP', 'Min', 'CS%', 'Save%.1', 'Cmp',
                     'Att', 'Cmp%', 'Att (GK)', 'Thr', 'Launch%', 'AvgLen', 'Att.1', 'Launch%.1', 'AvgLen.1', 'Stp%']

In [ ]:
correlations = []
years = []
for file in files:
    
    data_frame = pd.read_excel(file)

    data_frame.drop(columns=colonne_da_eliminare, inplace=True) 

    corr_matrix = data_frame.corr() 

    year = year_mapping[file] 
    years.append(year)

    fig = px.imshow(corr_matrix, title=f"Matrice di Correlazione - Anno {year}")

    fig.show()

Notiamo subito ottime correlazioni: le parate sono, ovviamente, ben correlate con i tiri in porta in quanto un portiere che subisce più tiri tendenzialmente dovrà fare più parate. Oltre a questo vediamo che è ben correlato con i clean sheet (partite concluse senza subire gol) infatti un portiere che fa molte parate ha più probabilità che concluda la partita senza concedere gol, ovviamente l'indice non è altissimo perchè se una squadra subisce molti tiri il portiere potrà fare tante parate e comunque concedere almeno un gol. Inoltre notiamo che i portieri che parano di più sono anche quelli che riescono ad intercettare più cross effettuati dagli avversari.

Un risultato abbastanza inaspettato è l'indice di correlazione fra parate e rigori parati. Ci aspettavamo che un portiere che fa tante parate parasse anche tanti rigori, invece scopriamo che l'indice è più basso di quanto pensavamo. Le due statistiche non sono completamente scorrelate, però l'indice fra queste non è molto alto, si aggira sullo 0.5 in media negli anni.

Abbiamo infine i dati su alcune tipologie specifiche di gol subiti. Gol subiti da punizione, gol subiti da calcio d'angolo e autogol. Fra questi tre vediamo tramite l'indice di correlazione che la maggior parte dei gol è subita da calcio d'angolo.

Per quanto riguarda i calci di rigore vediamo che l'indice fra quelli segnati e quelli parati abbastanza basso, ci suggerisce che parare i rigori sia una cosa tutt'altro che casuale ma basata esclusivamente sulle doti del portiere in questa particolare categoria. Se parare i rigori fosse completamente fortuna avremmo un indice più alto in quanto ad un numero di rigori concessi dovrebbe equivalere un numero simile di rigori parati. L'alto indice di correlazione fra i rigori calciati e segnati ci dice quanto sia raro trovare un portiere "para rigori".

In [ ]:
# Vediamo con i grafici i risultati appena trovati

stat1 = 'Saves'
stat2 = 'SoTA'
correlazione(stat1,stat2)

stat2 = 'CS'
correlazione(stat1,stat2)

stat2 = 'Stp'
correlazione(stat1,stat2)

stat2 = 'PKsv'
correlazione(stat1,stat2)

stat1 = 'PKA'
correlazione(stat1,stat2)

stat1 = 'GA'
stat2 = 'CK'
correlazione(stat1,stat2)

Passiamo ora alla seconda macro categoria, quella relativa ai passaggi ed i lanci.

In [9]:
colonne_da_eliminare =['Player', 'Nation', 'Pos', 'Squad','Age', 'MP', 'Starts', 'Min', 'GA', 'SoTA', 'Saves', 'Save%','AvgLen', 
'AvgLen.1','CS', 'CS%', 'PKatt', 'PKA', 'PKsv', 'PKm', 'Save%.1', 'FK', 'CK', 'OG', 'Cmp%', 'Launch%', 'Launch%.1', 'Opp', 'Stp', 
'Stp%']

In [ ]:
correlations = []
years = []
for file in files:
    
    data_frame = pd.read_excel(file)

    data_frame.drop(columns=colonne_da_eliminare, inplace=True) 

    corr_matrix = data_frame.corr() 

    year = year_mapping[file] 
    years.append(year)

    fig = px.imshow(corr_matrix, title=f"Matrice di Correlazione - Anno {year}")

    fig.show()

In queste matrici i colori possono trarre in inganno, in quanto vedendo un blu molto scuro si può pensare che ci siano delle statistiche mal correlate, ma non è così. Guardando, infatti, le barre di destra vediamo come il range di correlazione è sempre [0,8 ; 1] circa. Andando nello specifico vediamo che i passaggi tentati e completati hanno un indice alto, ciò comporta che i portieri in generale siano molto bravi con i piedi. Inoltre notiamo che il dato dei passaggi completati sia correlato bene con il dato dei lanci lunghi, la nostra conclusione è che un portiere fiducioso dei suoi piedi tenta spesso lanci lunghi.


In [ ]:
# Vediamo i grafici dei risultati appena trovati

stat1 = 'Cmp'
stat2 = 'Att'
correlazione(stat1,stat2)

stat2 = 'Thr'
correlazione(stat1,stat2)

stat2 = 'Att.1'
correlazione(stat1,stat2)